**INTRODUCCIÓN**

Una importante empresa extranjera tiene planes de construir su próximo hotel en el Ecuador, sin embargo esta empresa no está segura de cual es el lugar idoneo con un alto índice de plusvalía.

**DATOS**

Primero se necesita los datos del PIB en construcción que maneja cada provincia en el Ecuador, estos datos lo obtendremos del *Banco Central del Ecuador* en su página web.
https://www.bce.fin.ec/index.php/component/k2/item/293-cuentas-provinciales/

Despúes se necesita las coordenadas geográficas de las Capitales de cada provincia del Ecuadpr que lo encontramos en la siguiente página:
https://www.bce.fin.ec/index.php/component/k2/item/293-cuentas-provinciales/

In [1]:
# Importar librerias necesarias
import numpy as np
import pandas as pd
import json  
from geopy.geocoders import Nominatim
import requests
from pandas import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [2]:
# Explorar archivo csv con datos de las PIB de contruccion de las provincias del Ecuador
ecuador_data = pd.read_excel('Datos.xlsx')
ecuador_data

,PROVINCIA,PIB_CONST,CAPITAL,LATITUD,LONGITUD
0,GUAYAS,3573204.25,GUAYAQUIL,-2.1900,-79.8875
1,MANABI,843778.42,PORTOVIEJO,-1.0561,-80.4553
2,EL ORO,408806.26,MACHALA,-3.2667,-79.9667
3,LOS RIOS,371599.92,BABAHOYO,-1.8167,-79.5167
4,SANTA ELENA,371924.21,SANTA ELENA,-2.2267,-80.8583
5,ESMERALDAS,381322.17,ESMERALDAS,0.9667,-79.6528
6,PICHINCHA,2343166.09,QUITO,-0.2200,-78.5125
7,AZUAY,869822.87,CUENCA,-2.8974,-79.0045
8,TUNGURAHUA,247890.73,AMBATO,-1.2417,-78.6197
9,SANTO DOMINGO,464335.43,SANTO DOMINGO DE LOS COLORADOS,-0.2542,-79.1719


In [3]:
# Eliminar columnas innecesarias
ecuador_data = ecuador_data.drop(['PIB_CONST',	'CAPITAL'],axis=1)
ecuador_data

,PROVINCIA,LATITUD,LONGITUD
0,GUAYAS,-2.1900,-79.8875
1,MANABI,-1.0561,-80.4553
2,EL ORO,-3.2667,-79.9667
3,LOS RIOS,-1.8167,-79.5167
4,SANTA ELENA,-2.2267,-80.8583
5,ESMERALDAS,0.9667,-79.6528
6,PICHINCHA,-0.2200,-78.5125
7,AZUAY,-2.8974,-79.0045
8,TUNGURAHUA,-1.2417,-78.6197
9,SANTO DOMINGO,-0.2542,-79.1719


In [4]:
# Reduccion de la tabla a solo las 10 provincias del pais con mayor PIB en construcción del año 2019
principales_data = ecuador_data[ecuador_data.PROVINCIA.isin (['GUAYAS', 'PICHINCHA','AZUAY','MANABI','SANTO DOMINGO', 'LOJA', 'EL ORO', 'ESMERALDAS', 'SANTA ELENA','LOS RIOS'])]
principales_data

,PROVINCIA,LATITUD,LONGITUD
0,GUAYAS,-2.1900,-79.8875
1,MANABI,-1.0561,-80.4553
2,EL ORO,-3.2667,-79.9667
3,LOS RIOS,-1.8167,-79.5167
4,SANTA ELENA,-2.2267,-80.8583
5,ESMERALDAS,0.9667,-79.6528
6,PICHINCHA,-0.2200,-78.5125
7,AZUAY,-2.8974,-79.0045
9,SANTO DOMINGO,-0.2542,-79.1719
11,LOJA,-3.9833,-79.2000


In [5]:
# Obtner latitud y longitud de Ecuador
address = 'Ecuador'

geolocator = Nominatim(user_agent="ecuador_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Las coordenadas geograficas de Ecuador son {}, {}.'.format(latitude, longitude))

Las coordenadas geograficas de Ecuador son -1.3397668, -79.3666965.


In [6]:
# crear mapa de Ecuador utilizando los valores de latitud y longitud
map_ecuador = folium.Map(location=[latitude, longitude], zoom_start=10)

# añadir marcadores al mapa
for lat, lng, provincia in zip(principales_data['LATITUD'], principales_data['LONGITUD'], principales_data['PROVINCIA']):
    label = '{}'.format(provincia)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ecuador)  
    
#Se identifíca el lugar de las provincias del Ecuador con mas PIB en construcción
map_ecuador

In [7]:
# Se selecciona a la ciudad de Quito por ser el lugar más centrico del Ecuador 
# Se coloca Credenciales Foursquare
import requests
url="https://api.foursquare.com/v3/places/search"

headers = {
    "Accept": "kscamachoc",
    "Authorization": "fsq3Hj1OMjxDHh3QxZEA5VLaIRuIoHs4nxD1zAiQiNYM7UA="
}

response = requests.request("GET", url, headers=headers)

print(response.text)

{"results":[{"fsq_id":"4e07bcd8d22d658532a82358","categories":[{"id":16032,"name":"Park","short_name":"Park","plural_name":"Parks","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_","suffix":".png"}}],"chains":[],"closed_bucket":"VeryLikelyOpen","distance":906,"geocodes":{"main":{"latitude":-0.222129,"longitude":-78.499417}},"link":"/v3/places/4e07bcd8d22d658532a82358","location":{"address":"Jose Maria Aguirre N4-108","country":"EC","cross_street":"Concepción","formatted_address":"Jose Maria Aguirre N4-108 (Concepción), Quito","locality":"Quito","region":"Pichincha"},"name":"Parque Itchimbía","related_places":{},"timezone":"America/Guayaquil"},{"fsq_id":"555feefa498eb6f39b28a30d","categories":[{"id":13263,"name":"Japanese Restaurant","short_name":"Japanese","plural_name":"Japanese Restaurants","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/japanese_","suffix":".png"}}],"chains":[],"closed_bucket":"VeryLikelyOpen","distance":2243,"geocodes":{"m

In [8]:
# Se obtiene la Data de los lugares con mayor puntaje ya que estos disponen de una plusvalia alta
# Se procesa la Data para obtener los datos mas importantes
data_dict = json.loads(response.text)
names = []
categories = []
localities = []
regions = []
latitudes = []
longitudes = []
for result in data_dict.get("results", []):
    names.append(result.get("name", ""))
    category = result.get("categories", [{}])[0]
    categories.append(category.get("name", ""))
    location = result.get("location", {})
    localities.append(location.get("locality", ""))
    regions.append(location.get("region", ""))
    geocodes = result.get("geocodes", {}).get("main", {})
    latitudes.append(geocodes.get("latitude", ""))
    longitudes.append(geocodes.get("longitude", ""))

data = {
    'results.name': names,
    'results.categories.name': categories,
    'results.location.locality': localities,
    'results.location.region': regions,
    'results.geocodes.main.latitude': latitudes,
    'results.geocodes.main.longitude': longitudes
}
df = pd.DataFrame(data)

# Se muestra los lugares con mayor puntaje 
print(df)

              results.name   results.categories.name  \
0         Parque Itchimbía                      Park   
1                    Osaka       Japanese Restaurant   
2                  Shibumi          Asian Restaurant   
3            Noe Sushi Bar          Sushi Restaurant   
4  Lucía Pie House & Grill  Southern Food Restaurant   
5              Chez Jerome         French Restaurant   
6       Mirador De Guápulo            Scenic Lookout   
7                   Cyrano                    Bakery   
8        Casa de la Musica              Concert Hall   
9       Parque La Carolina                      Park   

  results.location.locality results.location.region  \
0                     Quito               Pichincha   
1                     Quito               Pichincha   
2                     Quito               Pichincha   
3                     Quito               Pichincha   
4                     Quito               Pichincha   
5     Hacienda Batán Grande               Pichincha  

In [9]:
# Se muestra los lugares con mayor puntaje
df

,results.name,results.categories.name,results.location.locality,results.location.region,results.geocodes.main.latitude,results.geocodes.main.longitude
0,Parque Itchimbía,Park,Quito,Pichincha,-0.222129,-78.499417
1,Osaka,Japanese Restaurant,Quito,Pichincha,-0.205391,-78.483586
2,Shibumi,Asian Restaurant,Quito,Pichincha,-0.192938,-78.499317
3,Noe Sushi Bar,Sushi Restaurant,Quito,Pichincha,-0.204338,-78.482126
4,Lucía Pie House & Grill,Southern Food Restaurant,Quito,Pichincha,-0.202972,-78.480830
5,Chez Jerome,French Restaurant,Hacienda Batán Grande,Pichincha,-0.196929,-78.481002
6,Mirador De Guápulo,Scenic Lookout,Quito,Pichincha,-0.201517,-78.479250
7,Cyrano,Bakery,Quito,Pichincha,-0.196456,-78.478531
8,Casa de la Musica,Concert Hall,Quito,Pichincha,-0.181225,-78.502037
9,Parque La Carolina,Park,Quito,Pichincha,-0.183506,-78.484440


In [10]:
# Se muestra en el mapa la ubicación de los lugares con mayor puntaje
# Se crea el mapa de Ecuador utilizando los valores de latitud y longitud
map_ecuador1 = folium.Map(location=[latitude, longitude], zoom_start=10)

# Se añaden marcadores al mapa
for lat, lng, nombre in zip(df['results.geocodes.main.latitude'], df['results.geocodes.main.longitude'], df['results.name']):
    label = '{}'.format(nombre)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ecuador1)  
    
map_ecuador1

**RESULTADOS**

- La mayor cantidad de los lugares se encuentran en el Norte de Quito.
- 2 lugares se encuentran en los parques más visitados de Quito
- 4 lugares se encuentran cerca de la Avenida Isabel Católica
- 6 lugares son establecimientos de comida

**DEBATE**

Claramente el sector donde se debe colocar el Hotel es en el Norte de Quito.
Ya que la mayoria de sitios estan en la Avenida Isabel Católica, se recomienda colocar el lugar cerca de esta Avenida. Caso contrario se debería colocar cerca del parque La Carolina.

**CONCLUSIONES**

- La provincia de Pichincha es la elegida para ser el lugar donde se contruira el Hotel por ser la más centrica de las provincias y la 2da con mayor PIB en construcción del Ecuador.
- El sector Norte de Quito tiene los establecimientos con mayor puntaje en la provincia de Pichincha.
- La Avenida Isabel Católica y el Parque La Carolina son los mejores para colocar en sus cercanias el Hotel de la empresa extranjera.